In [1]:
import pandas as pd 
import numpy as np
from collections import defaultdict

In [2]:
NEWS_USER_PATH = "../DatosKaggle/BuzzFeed/BuzzFeedNewsUser.csv"
USER_USER_PATH = "../DatosKaggle/BuzzFeed/BuzzFeedUserUser.txt"
NEWS_PATH = "news_id_title_type.csv"

In [3]:
df_news = pd.read_csv(NEWS_PATH)
df_news.index = df_news.index + 1 
df_news.head()

,id,title,news_type
1,1,Another Terrorist Attack in NYC…Why Are we STI...,Real
2,2,Hillary Clinton on police shootings: 'too many...,Real
3,3,"Critical counties: Wake County, NC, could put ...",Real
4,4,NFL Superstar Unleashes 4 Word Bombshell on Re...,Real
5,5,Obama in NYC: 'We all have a role to play' in ...,Real


In [4]:
df_news_user = pd.read_csv(NEWS_USER_PATH)
df_news_user.head(n=2)

,news_id,user,share_freq
0,45,1,1
1,127,2,1


In [5]:
df_user_user = pd.read_table(USER_USER_PATH, sep="\t", names=['id_source', 'id_target'])
df_user_user.head(n=2)

,id_source,id_target
0,48,1
1,899,1


In [6]:
df_user_user.loc[df_user_user['id_source'] == 48]

,id_source,id_target
0,48,1
776,48,6
834,48,8
992,48,12
1225,48,17
...,...,...
628648,48,15125
631692,48,15199
632757,48,15209
633617,48,15221


Creamos una lista de los usuarios a los que cada usuario sigue

In [7]:
user_to_following = defaultdict(list)

for index, row in df_user_user.iterrows():
    id_source = row["id_source"]
    id_target = row["id_target"]
    user_to_following[id_source].append(id_target)   

Validacion de resultados: Comparamos el dataset original con los resultados de user_to_following

In [8]:
df_news_user.head()

,news_id,user,share_freq
0,45,1,1
1,127,2,1
2,115,3,1
3,180,3,1
4,140,4,1


In [9]:
df_news_user["following"] = ""
user_id_to_news_type = defaultdict(list)
news_type = []
for index, row in df_news_user.iterrows():
    news_type_tmp = df_news.news_type.loc[row.news_id]
    user_id_to_news_type[row.user].append((news_type_tmp, row.share_freq))
    df_news_user.at[index, 'following'] = user_to_following.get(row['user'], [])
    news_type.append(news_type_tmp)


df_news_user['news_type'] = news_type
df_news_user.head()

,news_id,user,share_freq,following,news_type
0,45,1,1,[],Real
1,127,2,1,"[100, 199, 410, 621, 664, 1375, 1432, 1506, 17...",Fake
2,115,3,1,[],Fake
3,180,3,1,[],Fake
4,140,4,1,"[14, 36, 52, 65, 74, 95, 108, 117, 141, 208, 2...",Fake


Filtramos el dataset deduplicando el user id

In [10]:
df_news_following = df_news_user[['user', 'following']]
df_news_following = df_news_following.drop_duplicates(subset=['user']).sort_values(by=['user'])
df_news_following.head()

,user,following
0,1,[]
1,2,"[100, 199, 410, 621, 664, 1375, 1432, 1506, 17..."
2,3,[]
4,4,"[14, 36, 52, 65, 74, 95, 108, 117, 141, 208, 2..."
5,5,[9220]


In [11]:
def count_fake_news(following: list):
    count = 0
    for user in following:
        news_type_freq = user_id_to_news_type.get(user)
        for news in news_type_freq:
            if news[0] == 'Fake':
                count += news[1]
    return count
        
def count_real_news(following: list):
    count = 0
    for user in following:
        news_type_freq = user_id_to_news_type.get(user)
        for news in news_type_freq:
            if news[0] == 'Real':
                count += news[1]
    return count

def count_total_news(following: list):
    count = 0
    for user in following:
        news_type_freq = user_id_to_news_type.get(user)
        for news in news_type_freq:
            count += 1
    return count
    
def count_total_freq_news(following: list):
    count = 0
    for user in following:
        news_type_freq = user_id_to_news_type.get(user)
        for news in news_type_freq:
            count += news[1]
    return count
    
    
df_news_following['fake_news_shared_by_followees'] = df_news_following["following"].apply(lambda x : count_fake_news(x))
df_news_following['real_news_shared_by_followees'] = df_news_following["following"].apply(lambda x : count_real_news(x))
df_news_following['total_news_shared_by_followees'] = df_news_following["following"].apply(lambda x : count_total_news(x))
df_news_following['total_freq_news_shared_by_followees'] = df_news_following["following"].apply(lambda x : count_total_freq_news(x))

In [12]:
df_news_following.head()

,user,following,fake_news_shared_by_followees,real_news_shared_by_followees,total_news_shared_by_followees,total_freq_news_shared_by_followees
0,1,[],0,0,0,0
1,2,"[100, 199, 410, 621, 664, 1375, 1432, 1506, 17...",50,56,100,106
2,3,[],0,0,0,0
4,4,"[14, 36, 52, 65, 74, 95, 108, 117, 141, 208, 2...",607,576,1067,1183
5,5,[9220],0,1,1,1


In [13]:
df_news_following["count_following"] = df_news_following["following"].apply(lambda x : len(x))

In [14]:
df_news_following.tail()

,user,following,fake_news_shared_by_followees,real_news_shared_by_followees,total_news_shared_by_followees,total_freq_news_shared_by_followees,count_following
22765,15253,"[4315, 6808]",1,1,2,2,2
22766,15254,"[209, 3983, 4864, 6802, 10062, 11319, 11457, 1...",10,3,13,13,8
22767,15255,[],0,0,0,0,0
22768,15256,[1423],1,0,1,1,1
22769,15257,"[4678, 5765, 6370, 8765, 10436, 13151]",4,3,7,7,6


Agregamos una columna llamada fake. Si la variable fake_news_shared_by_followees toma valores distintos a 0, entonces fake vale 1 (diciendonos que algunos de los usuarios a los que un determinado user sigue, compartio una fake news)

In [ ]:
df_news_following["fake"] = df_news_following["fake_news_shared_by_followees"].apply(lambda x : 1 if x > 0 else 0)
df_news_following["fake_ratio"] = (df_news_following["fake_news_shared_by_followees"] / df_news_following["total_freq_news_shared_by_followees"]).replace(np.nan, 0)
df_news_following["fake"] = df_news_following["fake_ratio"].apply(lambda x : 1 if x > 0 else 0)
df_news_following.head()

In [ ]:
df_news_following.to_csv("../DatosKaggle/BuzzFeed/sample/NewsFollowing_20210815.csv", index = False, encoding='utf-8-sig')

## Prediction Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
X = df_news_following[['count_following']]
y = df_news_following["fake"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
clf = RandomForestClassifier()

clf = clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()

logisticRegr.fit(X_train, y_train)

y_pred = logisticRegr.predict(X_test)

In [ ]:
X_test['predicted_proba'] = logisticRegr.predict_proba(X_test)[:,1]

In [ ]:
logisticRegr.predict_proba(X_test)

In [ ]:
X_test

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)